In [ ]:
!pip uninstall ndvi_tools -y

In [ ]:
!pip install ndvi_tools/

In [ ]:
import os
import json
import xarray as xr
import geopandas as gpd

import datacube
import matplotlib.pyplot as plt
from deafrica_tools.plotting import display_map
from deafrica_tools.dask import create_local_dask_cluster

from ndvi_tools.ndvi_climatology_plugin import NDVIAnomaly

## Test plugin without odc-stats

In [ ]:
create_local_dask_cluster()

In [ ]:
dc = datacube.Datacube(app="Vegetation_anomalies")

In [ ]:
lat, lon = 9.8892,-0.9271#31.0004, 29.6856#6.51010,-2.7352#13.6683,-0.99460
lat_buffer = 0.15
lon_buffer = 0.15
# Set the range of dates for the climatology
time_range = ('2022-01')
resolution = (-30, 30)

lat_range = (lat-lat_buffer, lat+lat_buffer)
lon_range = (lon-lon_buffer, lon+lon_buffer)

In [ ]:
# display_map(x=lon_range, y=lat_range)

In [ ]:
query = {
    'x': lon_range,
    'y': lat_range,
    'time': time_range,
    'resolution': resolution,
    'output_crs':'epsg:6933',
    'measurements':['red','nir','pixel_quality'],
    'group_by':'solar_day',
#     'collection_category':'T1'
#     'dask_chunks':dask_chunks
}

### grab data so we can use the geobox parameter to mimic a 'task'

In [ ]:
task_ls = dc.load(product=['ls8_sr', 'ls9_sr'], dask_chunks={}, **query)

In [ ]:
task

### grab lists of datatsets to mimic .db files

In [ ]:
dss = dc.find_datasets(product=['ls8_sr', 'ls9_sr'], **query)

### set up an example config

In [ ]:
# config = dict(
#     resampling="bilinear",
#     bands=["red", "nir", "green", "blue"],
#     mask_band="QA_PIXEL",
#     harmonization_slope=0.988,
#     harmonization_intercept= -0.015,
#     rolling_window = 3,
#     mask_filters=[["opening", 5], ["dilation", 5]],
#     work_chunks= dict(x=1900, y=1900),
#     flags_ls57=dict(cloud="high_confidence", cloud_shadow="high_confidence"),
#     flags_ls8=dict(
#         cloud="high_confidence",
#         cloud_shadow="high_confidence",
#         cirrus="high_confidence",
#     ),
# )

### Run the plugin code

In [ ]:
x=NDVIClimatology(**config)
ndvi = x.input_data(dss, task.geobox)
result = x.reduce(ndvi)#.compute()
print(result)

In [ ]:
%%time
result = result.compute()

### Plot

# Test plugin with odc-stats

In [ ]:
!pip uninstall ndvi_tools -y

In [ ]:
!pip install ndvi_tools/

In [ ]:
import os
import json
import xarray as xr
import geopandas as gpd
from odc.stats.tasks import TaskReader
from odc.stats.model import OutputProduct

### Generate tasks etc

In [ ]:
%%time
!odc-stats save-tasks --grid africa_30 --temporal-range 2022-01--P1M ls8_sr-ls9_sr-s2_l2a-ndvi_climatology_ls #--dataset-filter '{"collection_category": "T1"}'

In [ ]:
# os.system("cm-task --task-csv=ls5_sr-ls7_sr-ls8_sr_2011--P5Y.csv "\
#           "--geojson=../testing/data/testing_extent.geojson "\
#           "--outfile=aez.csv "\
#           "--grid=africa_30 " \
#           "--sqs=deafrica-dev-eks-stats-ndvi-clim "\
#           "--db=ls5_sr-ls7_sr-ls8_sr_2011--P5Y.db "\
#          )

### Enter the X and Y Tile ID of the tile you want to run

Use the geojson to select a tile id

In [ ]:
tile_x = '178' #x173y118 x179/y094
tile_y = '115'  	

t=[int(tile_x), int(tile_y)]

### Extract the tasks object for that tile

In [ ]:
# product_dss = {}
# for dataset in taskdb.datasets((t[0], t[1])):
#     product = get_in(["product", "name"], dataset.metadata_doc)
#     if product not in product_dss:
#         product_dss[product] = []
#     product_dss[product].append(dataset)

In [ ]:
name, version = 'ndvi_anomaly', '1-0-0'
op = OutputProduct(
            name='ndvi_anomaly_ls',
            version='1-0-0',
            short_name='ndvi_anomaly_ls',
            location=f"s3://dummy-bucket/{name}/{version}",
            properties={"odc:file_format": "GeoTIFF"},
            measurements=['red']
        )

taskdb = TaskReader('ls8_sr-ls9_sr-s2_l2a-ndvi_climatology_ls_2022-01--P1M.db', product=op)
task = taskdb.load_task(('2021--P2Y', t[0], t[1]))

### Optionally export tile geojson to view

In [ ]:
# with open('task_tile_check.geojson', 'w') as fh:
#     json.dump(task.geobox.extent.to_crs('epsg:4326').json, fh, indent=2)

### Find the index of the tile we want to run

In [ ]:
tile_index_to_run = []
all_tiles = list(taskdb.all_tiles)
for i, index in zip(all_tiles, range(0, len(all_tiles))):
    if (i[1]==t[0]) & (i[2]==t[1]):
        tile_index_to_run.append(index)
        print(index)

### Try running odc-stats using the config yaml and external plugin

In [ ]:
%%time
os.system("odc-stats run "\
          "ls8_sr-ls9_sr-s2_l2a-ndvi_climatology_ls_2022-01--P1M.db "\
          "--config=///home/jovyan/git/ndvi-anomalies/production/ndvi_tools/config/ndvi_anomaly.yaml "\
          "--resolution=120 "\
          "--threads=4 "\
          "--memory-limit=25Gi "\
          "--location=file:///home/jovyan/git/ndvi-anomalies/production/{product}/{version}/ "+str(tile_index_to_run[0])
         )

In [ ]:
# !odc-stats run ls7_sr-ls8_sr_2014--P2Y.db --config=ndvi_climatology.yaml --resolution=80 --threads=4 --memory-limit=29Gi --location=file:///home/jovyan/git/deafrica-sandbox-notebooks/frica-sandbox-notebooks/HLS/{product}/{version} 1202

### Plot results

### remove results folder

In [ ]:
!rm -r -f ndvi_anomaly/

## Sync folder from s3

In [ ]:
!aws s3 sync s3://deafrica-data-dev-af/ndvi_anomaly ndvi_anomaly  --no-sign-request